In [1]:
#from parse_kd import *
import re
import pandas as pd
import numpy as np
from chunk_utils import *
import matplotlib
import scipy.cluster.hierarchy as hc
import matplotlib.pyplot as plt
from pyclustering.cluster.xmeans import xmeans
data_dir = './data/dw_freetext/'

In [2]:
%load_ext autoreload
%autoreload 2
matplotlib.rcParams['figure.figsize'] = 8,5

In [3]:
df = pd.read_csv(data_dir+'GunControl_400.csv', delimiter='\t', usecols=[0,6,10])
df_filtered = df[df['Task']=='Copy_2'].reset_index(drop=True)
# "Copy_2" column = when they're stating true opinion, repeating (transcribing) the essay already written
# resrt_index() is essential!

new_uids = pd.Series(map(lambda x: "user"+str(x), range(df_filtered.shape[0])))
df_filtered = df_filtered.assign(uid=new_uids)

df_filtered.head(7)

,UserName,Task,ReviewMeta,uid
0,A10E8QHPV30BAD,Copy_2,0 MouseUp 0 0;874 KeyDown 16;946 KeyDown 73;10...,user0
1,A11T1LGIWPM9Z1,Copy_2,0 MouseUp 0 0;784 KeyDown 16;863 KeyDown 84;92...,user1
2,A145PJ769UGAID,Copy_2,0 MouseUp 0 0;1517 KeyDown 16;1517 KeyDown 87;...,user2
3,A161NZP385W1TX,Copy_2,0 MouseUp 0 0;1621 KeyDown 16;1734 KeyDown 84;...,user3
4,A168RQT8B18FH,Copy_2,0 MouseUp 0 0;2112 KeyDown 16;2311 KeyDown 73;...,user4
5,A16HFBY06FP1RJ,Copy_2,0 MouseUp 0 0;317 KeyDown 16;543 KeyDown 71;62...,user5
6,A191P01GHJRPWO,Copy_2,0 MouseUp 0 0;7007 KeyDown 16;7018 KeyDown 73;...,user6


In [4]:
phrases = ["criminal", "constitution", "amendment", "fundamental", "right", "gun", "control", 
           "safe", "citizen", "kill", "violence", "strongly", "think", "feel", 
           "legal", "reason", "fear", "oppose", "government", "power", 
           "freedom", "people"
          ]

# dict of dicts -- examining at user level
userwise_delays = {}
# dict of lists --- examining at population level (throwing away user information)
phrasewise_delays = {x:[] for x in phrases}

for textIndex in range(df_filtered.shape[0]):
#for textIndex in range(80,130):    
    rawkd = df_filtered['ReviewMeta'].iloc[textIndex]
    uid = df_filtered['uid'].iloc[textIndex]
    userwise_delays[uid] = {}
    
    for phrase in phrases:
        phrase_keys = get_phrases_from_rawkd(rawkd, phrase)    
        all_delays = [keypress2delays(phrase, keys) for keys in phrase_keys]
        userwise_delays[uid][phrase] = all_delays
        phrasewise_delays[phrase].extend(all_delays)

<b> 

pick a word

represent each user by a vector: the average delay vector of that word

discover clusters of users .. separately for each word

compare the clusters over different words

<b>

In [5]:
#clusters = 
for word_of_interest in phrases:
    userwise_avgs = {}
    print ("\n\n"+word_of_interest)
    for uid in userwise_delays:
        if len(userwise_delays[uid][word_of_interest]):
            word1_avg = np.average(userwise_delays[uid][word_of_interest], axis=0)
            userwise_avgs[uid] = word1_avg
    
    uid_keys = list(userwise_avgs.keys())
    
    if len(uid_keys):
        vals = np.array(list(userwise_avgs.values()))
        clus = xmeans(vals, tolerance=3)
        # tolerance(double): Stop condition for each iteration: if maximum value of change of centers of clusters is less than tolerance than algorithm will stop processing. 
        clus.process()
        clusterids = clus.get_clusters()
    
        for i in range(len(clusterids)):
            cluster_n_uids = [uid_keys[x] for x in clusterids[i]]
            print (cluster_n_uids, " ---> cluster ", i)



criminal
['user0', 'user3', 'user4', 'user14', 'user24', 'user38', 'user43', 'user44', 'user49', 'user55', 'user56', 'user60', 'user67', 'user72', 'user75', 'user76', 'user78', 'user81', 'user90', 'user91', 'user94', 'user97', 'user100', 'user102', 'user107', 'user108', 'user109', 'user114', 'user115', 'user121', 'user126', 'user133', 'user134', 'user142', 'user144', 'user153', 'user154', 'user169', 'user170', 'user172', 'user173', 'user182', 'user185', 'user188', 'user191', 'user193', 'user199', 'user201', 'user206', 'user210', 'user211', 'user214', 'user215', 'user216', 'user222', 'user223', 'user225', 'user226', 'user230', 'user237', 'user238', 'user248', 'user250', 'user252', 'user259', 'user260', 'user261', 'user263', 'user267', 'user269', 'user270', 'user271', 'user272', 'user273', 'user274', 'user276', 'user279', 'user282', 'user283', 'user284', 'user290', 'user292', 'user294', 'user299', 'user301', 'user302', 'user306', 'user315', 'user316', 'user325', 'user328', 'user330', '